In [3]:
import torch
import torch.nn as nn
import torchvision as tv

import numpy as np
from pathlib import Path

In [56]:
def get_mnist_ds(is_train:bool):
    return tv.datasets.MNIST(root = Path('mnistdata'),
                             train = is_train,
                             transform = tv.transforms.ToTensor(),
                             target_transform = None,
                             download = True
                            )

In [67]:
def get_mnist_ds_loader(batch_size):
        train_ds = get_mnist_ds(is_train = True)
        valid_ds = get_mnist_ds(is_train = False)
        
        return (torch.utils.data.DataLoader(train_ds, batch_size, shuffle = True),
                torch.utils.data.DataLoader(valid_ds, 2*batch_size, shuffle = False))
    

In [69]:
train_dl, valid_dl = get_mnist_ds_loader(32)

In [75]:
len(train_dl.dataset), len(valid_dl.dataset)

(60000, 10000)